## Import libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [ ]:
data = np.load(npz_file)
images = data['face_images']  # assuming images are stored under 'images' key
print(np.shape(images))
images = np.transpose(images, (2, 0, 1))
print(np.shape(images))
coordinates = np.array(pd.read_csv(csv_file).values)
print(np.shape(coordinates))

keep_ix = np.logical_not(np.isnan(coordinates[:,:4])).all(axis=1)
images = images[keep_ix,:]
coordinates = coordinates[keep_ix,:4]
print(np.shape(images))
print(np.shape(coordinates))

for j in range(np.shape(images)[0]):
    images[j,:] = (images[j,:] - images[j,:].mean()) / images[j,:].std()

## Load data

In [5]:
# Load and preprocess data
npz_file = '../dataset/face_images.npz'
csv_file = '../dataset/facial_keypoints.csv'

data = np.load(npz_file)
images = data['face_images']  # assuming images are stored under 'images' key
coordinates = np.array(pd.read_csv(csv_file).values)

# Reshape image data
images = np.transpose(images, (2, 0, 1))

# Find samples that have at coordinates for centres of both eyes
keep_ix = np.logical_not(np.isnan(coordinates[:,:4])).all(axis=1)
images = images[keep_ix,:]
coordinates = coordinates[keep_ix,:4]

# Normalize images
for j in range(np.shape(images)[0]):
    images[j,:] = (images[j,:] - images[j,:].mean()) / images[j,:].std()

# Reshape images to (num_samples, 1, height, width) if necessary
if len(images.shape) == 3:
    images = images[:, np.newaxis, :, :]

# Convert to PyTorch tensors
images_tensor = torch.tensor(images, dtype=torch.float32)
coordinates_tensor = torch.tensor(coordinates, dtype=torch.float32)

# Split the data
train_images, test_images, train_coords, test_coords = train_test_split(
    images_tensor, coordinates_tensor, test_size=0.2, random_state=42)

train_dataset = TensorDataset(train_images, train_coords)
test_dataset = TensorDataset(test_images, test_coords)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

## Create Convolutional Neural Network model

In [27]:
# 3. Create a convolutional neural network
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128, 4)  # assuming input images are 32x32

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv3(x))
        x = x.mean(3).mean(2) # Global average pooling
        x = torch.relu(self.fc1(x))
        return x

model = CNN()

## Train the model on the data

In [28]:
# Train the model
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 20
train_losses = []
test_losses = []

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for images, coords in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, coords)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
    
    train_loss = train_loss / len(train_loader.dataset)
    train_losses.append(train_loss)
    
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for images, coords in test_loader:
            outputs = model(images)
            loss = criterion(outputs, coords)
            test_loss += loss.item() * images.size(0)
    
    test_loss = test_loss / len(test_loader.dataset)
    test_losses.append(test_loss)
    
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')

Epoch 1/20, Train Loss: 14.9508, Test Loss: 11.6528
Epoch 2/20, Train Loss: 11.4820, Test Loss: 11.4142
Epoch 3/20, Train Loss: 11.2811, Test Loss: 11.1386
Epoch 4/20, Train Loss: 11.1479, Test Loss: 11.0488
Epoch 5/20, Train Loss: 11.0888, Test Loss: 11.3766
Epoch 6/20, Train Loss: 11.0648, Test Loss: 11.0752
Epoch 7/20, Train Loss: 11.0339, Test Loss: 11.0248
Epoch 8/20, Train Loss: 11.0545, Test Loss: 10.9632
Epoch 9/20, Train Loss: 11.0344, Test Loss: 11.0483
Epoch 10/20, Train Loss: 11.0128, Test Loss: 11.0987
Epoch 11/20, Train Loss: 11.0214, Test Loss: 10.9662
Epoch 12/20, Train Loss: 11.0490, Test Loss: 10.9539
Epoch 13/20, Train Loss: 11.0264, Test Loss: 10.9399
Epoch 14/20, Train Loss: 11.0179, Test Loss: 11.0064
Epoch 15/20, Train Loss: 11.0098, Test Loss: 11.1594
Epoch 16/20, Train Loss: 11.0363, Test Loss: 10.9510
Epoch 17/20, Train Loss: 10.9814, Test Loss: 10.9366
Epoch 18/20, Train Loss: 11.0264, Test Loss: 11.0022
Epoch 19/20, Train Loss: 11.0359, Test Loss: 10.9736
Ep

## Plot results from the model

In [3]:
# Plot the losses
plt.figure(figsize=(10,5))
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()
## MOST RECENT

NameError: name 'train_losses' is not defined

<Figure size 1000x500 with 0 Axes>